# Introduction

Used to extract the harmonic frequency calculation data from the output. Returned to a Thermal_output class, which is inherited from Crystal_out, with attributes of crystal output objects and thermodynamic-specific attributes, including:

* self.lattice, pymatgen structure objection, for geometry information.  
* self.nqpoint, int, number of q point where the frequencies are calculated.  
* self.qpoint, nqpoint * 3 numpy array, the fractional coordinates of q points in reciprocal lattice.  
* self.nmode, nqpoint * 1 numpy array, the numbers of modes at each q point.  
* self.frequency, nqpoint * nmode numpy array, the vibrational frequencies ($\nu$) of each mode at each q point. Unit: THz. 

Note: 

Angular frequency: $\omega = 2\pi\nu$  
Eigen value of dynamic matrix: $\lambda = \nu^{2}$

* self.eigenvector, nqpoint * nmode * natom * 3 numpy array, the eigenvectors of dynamic matrix at each atom, each vibrational mode and each qpoint. Classical amplitude (298.15K), expressed in Cartesian coordinates. Unit: Angstrom.


# Examples
    
1. Form I paracetamol, 'freqf1-r0.out', $\Gamma$ point HA frequencies. nqpoint = 1, nmode = 240, natom = 80.  
2. A graphene primitive cell, 'nostr-modes.out', phonon dispersion calculated along the $\Gamma-K-M$ path, with occasional warning messages. nqpoint = 123, nmode = 6, natom = 2.

In [17]:
from crystal_functions.file_readwrite import Crystal_output
from crystal_functions.convert import cry_out2pmg


class Thermal_output(Crystal_output):
    """
    Class Thermal_output, inheriated from Crystal_out and with thermodynamic-
    specific attributes, including:
        self.lattice: __init__, Lattice information
        self.nqpoint: get_qpoint, Number of q points
        self.qpoint: get_qpoint, Fractional coordinates of qpoints
        self.nmode: get_mode, Number of vibrational modes at all qpoints
        self.frequency: get_mode, Frequencies of all modes at all qpoints, Unit: THz
        self.eigenvector: get_eigenvector, Eigenvectors (classical amplitude) of 
                          all atoms, all modes at all qpoints. Unit: Angstrom
    """
    def __init__(self, output_name):
        """
        Input:
            The name of '.out' file
        Output:
            self, Crystal_functions output object
            self.lattice, pymatgen structure object, lattice and atom information
        """
        super(Thermal_output, self).__init__(output_name)
        self.lattice = cry_out2pmg(self, initial=False, vacuum=500)

    def get_qpoint(self):
        """
        Get the qpoints at which the phonon frequency is calculated.
        Input:
            -
        Output:
            self.nqpoint, int, Number of q points where the frequencies are calculated.
            self.qpoint, nq * 3 numpy float array, fractional coordinates of qpoints.
        """
        import numpy as np
        import re
        
        self.nqpoint = 0
        self.qpoint = np.array([], dtype=float)

        for i, line in enumerate(self.data):
            if re.search('EXPRESSED IN UNITS        OF DENOMINATOR', line):
                shrink = int(line.strip().split()[-1])
                
            if re.search('DISPERSION K POINT NUMBER', line):
                coord = np.array(line.strip().split()[7:10], dtype=float)
                self.qpoint = np.append(self.qpoint, coord / shrink)
                self.nqpoint += 1
        
        self.qpoint = np.reshape(self.qpoint, (-1, 3))
        if self.nqpoint == 0:
            self.nqpoint = 1
            self.qpoint = np.array([0, 0, 0], dtype=float)
            
        return self.nqpoint, self.qpoint

    def get_mode(self):
        """
        Get corresponding vibrational frequencies and for all modes and
        compute the total number of vibration modes (natoms * 3).

        Input:
            -
        Output:
            self.nmode, nqpoint * 1 numpy int array, Number of vibration modes at each
                        qpoints.
            self.frequency: nqpoint * nmode numpy float array, Harmonic vibrational
                        frequency. Unit: THz
        """
        import numpy as np
        import re
        
        if not hasattr(self, 'nqpoint'):
            self.get_qpoint()
        
        self.frequency = np.array([], dtype=float)

        countline = 0
        while countline < len(self.data):
            is_freq = False
            if re.search('DISPERSION K POINT NUMBER', self.data[countline]):
                countline += 2
                is_freq = True
            
            if re.search('MODES         EIGV          FREQUENCIES     IRREP',
                         self.data[countline]):
                countline += 2
                is_freq = True

            while self.data[countline].strip() and is_freq:
                line_data = self.ignore_signal_line(self.data[countline])
                nm_a = int(line_data[0].strip('-'))
                nm_b = int(line_data[1])
                freq = float(line_data[4])

                for mode in range(nm_a, nm_b + 1):
                    self.frequency = np.append(self.frequency, freq)

                countline += 1
                
            countline += 1

        self.frequency = np.reshape(self.frequency, (self.nqpoint, -1))
        self.nmode = np.array([len(i) for i in self.frequency], dtype=float)

        return self.nmode, self.frequency

    def get_eigenvector(self):
        """
        Get corresponding mode eigenvectors for all modes on all
        atoms in the supercell. 
        
        Input:
            -
        Output:
            self.eigenvector, nqpoint * nmode * natom * 3 numpy float array, 
                              Eigenvectors expressed in Cartesian coordinate,
                              at all atoms, all modes and all qpoints. Classical
                              amplitude. Unit: Angstrom
        """
        import numpy as np
        import re
        
        if not hasattr(self, 'nmode'):
            self.get_mode()
        
        total_mode = np.sum(self.nmode)
        countline = 0
        # Multiple blocks for 1 mode. Maximum 6 columns for 1 block.
        if np.max(self.nmode) >= 6:
            countmode = 6
        else:
            countmode = total_mode
        
        # Read the eigenvector region as its original shape
        block_label = False
        total_data = []
        while countline < len(self.data) and countmode <= total_mode:
            # Gamma point / phonon dispersion calculation
            if re.match(r'^ MODES IN PHASE', self.data[countline]) or\
               re.match(r'^ NORMAL MODES NORMALIZED', self.data[countline]):
                block_label = True
            elif re.match(r'^ MODES IN ANTI-PHASE', self.data[countline]):
                block_label = False

            # Enter a block
            if 'FREQ(CM**-1)' in self.data[countline] and block_label:
                countline += 2
                block_data = []
                while self.data[countline].strip():
                    # Trim annotation part (12 characters)
                    line_data = self.ignore_signal_line(self.data[countline][13:])
                    if line_data:
                        block_data.append(line_data)

                    countline += 1

                countmode += len(line_data)
                total_data.append(block_data)

            countline += 1

        total_data = np.array(total_data, dtype=float)

        # Rearrage eigenvectors
        block_per_q = len(total_data) / self.nqpoint
        self.eigenvector = []
        # 1st dimension, nqpoint
        for q in range(self.nqpoint):
            index_bg = int(q * block_per_q)
            index_ed = int((q + 1) * block_per_q)
            q_data = np.hstack([i for i in total_data[index_bg : index_ed]])
        # 2nd dimension, nmode    
            q_data = np.transpose(q_data)
        # 3rd dimension, natom
            natom = len(self.lattice.sites)
#             natom = int(self.nmode[0] / 3)
            q_rearrange = [np.split(m, natom, axis=0) for m in q_data]
            
            self.eigenvector.append(q_rearrange)

        self.eigenvector = np.array(self.eigenvector) * 0.5292

        return self.eigenvector
    
    def ignore_signal_line(self, text, split_mark=''):
        """
        Not a function for thermodynamics. Ignore the signalling floating-point
        exceptions in the output. Used for multi-line data. Warning message:
            Note: The following floating-point exceptions are signalling:
        
        Input: 
            A text line generated by output.readlines()
        Output:
            The split line as a list of strings if this line does not begin
            with 'Note:' / 'IEEE'
        """
        if 'Note:' in text or 'IEEE_' in text:
            return
        
        if not split_mark:
            return text.strip().split()
        else:
            return text.strip().split(split_mark)
            

# Tests
## Initialization and geometry

Note: Getting primitive lattice vectors of 2D and 1D periodic systems is under developing when this example is written. Thermal_output object of graphene is generated by avoide using `self.lattice` related commands. Line 26 and 178 were commented and line 179 was used to generate the 'paracetamol' and 'graphene' objects, except the cell below. 

In [5]:
paracetamol = Thermal_output('freqf1-r0.out')
print(paracetamol.lattice)

Full Formula (H36 C32 N4 O8)
Reduced Formula: H9C8NO2
abc   :   7.163003   9.276183  12.670481
angles:  90.000000 116.369705  90.000000
Sites (80)
  #  SP            a          b          c
---  ----  ---------  ---------  ---------
  0  O      0.055848   0.430591  -0.28671
  1  O     -0.055848  -0.069409  -0.21329
  2  O     -0.055848  -0.430591   0.28671
  3  O      0.055848   0.069409   0.21329
  4  O      0.453954  -0.00189    0.302573
  5  O     -0.453954   0.49811    0.197427
  6  O     -0.453954   0.00189   -0.302573
  7  O      0.453954  -0.49811   -0.197427
  8  N     -0.212002   0.043338   0.340211
  9  N      0.212002  -0.456662   0.159789
 10  N      0.212002  -0.043338  -0.340211
 11  N     -0.212002   0.456662  -0.159789
 12  C     -0.149584   0.138119   0.436523
 13  C      0.149584  -0.361881   0.063477
 14  C      0.149584  -0.138119  -0.436523
 15  C     -0.149584   0.361881  -0.063477
 16  C     -0.237603   0.142163  -0.485327
 17  C      0.237603  -0.357837  -0.0146

## Get qpoints and frequencies at each qpoint

nqpoint = 1, nmode = 240 for paracetamol case, nqpoint = 123, nmode = 3 for graphene case. In both cases, nqpoint should be an integer and nmode should be a numpy array.

In [19]:
paracetamol = Thermal_output('freqf1-r0.out')
graphene = Thermal_output('nostr_modes.out')

paracetamol.get_mode()
graphene.get_mode()

print('Paracetamol qpoint:', paracetamol.nqpoint, paracetamol.qpoint)
print('Graphene qpoints:', graphene.nqpoint)

print('Paracetamol modes:', len(paracetamol.nmode), paracetamol.nmode)
print('Graphene modes:', len(graphene.nmode), graphene.nmode)

Paracetamol qpoint: 1 [0. 0. 0.]
Graphene qpoints: 123
Paracetamol modes: 1 [240.]
Graphene modes: 123 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6. 6.
 6. 6. 6.]


## Get eigenvectors
Self.eigenvector should be a 1 (nqpoint) * 240 (nmode) * 80 (natom) * 3 (xyz) numpy array for paracetamol case. A 123 * 6 * 2 * 3 array for graphene case.

In [21]:
paracetamol.get_eigenvector()
graphene.get_eigenvector()

print('Paracetamol')
print('nqpoint=', len(paracetamol.eigenvector),
      'nmode=', len(paracetamol.eigenvector[0]),
      'natom=', len(paracetamol.eigenvector[0, 0]),
      'first eigenvector=', paracetamol.eigenvector[0, 0, 0])

print('Graphene')
print('nqpoint=', len(graphene.eigenvector),
      'nmode=', len(graphene.eigenvector[0]),
      'natom=', len(graphene.eigenvector[0, 0]),
      'first eigenvector=', graphene.eigenvector[0, 0, 0])


Paracetamol
nqpoint= 1 nmode= 240 natom= 80 first eigenvector= [ 0.0214326  0.        -0.0021168]
Graphene
nqpoint= 123 nmode= 6 natom= 2 first eigenvector= [ 0.         -0.          0.10800972]


Example: Find the qpoint coordinate, frequency and eigenvectors of all atoms in the graphene primitive cell, with a given qpoint (75) and vibrational mode (3rd)

In [24]:
print('Wavevector coordinates:', graphene.qpoint[74])
print('Frequency:', graphene.frequency[74, 2])
print('Eigenvectors:', graphene.eigenvector[74, 2])

Wavevector coordinates: [0.47083333 0.05833333 0.        ]
Frequency: 19.8656
Eigenvectors: [[ 5.2920000e-05  1.0795680e-01 -5.2920000e-05]
 [-5.2920000e-04  1.0615752e-01  5.2920000e-05]]


Compare with the crystal output file:

At the start point of $K-M$ path:  
```
 *******************************************************************************

  PHONONS ALONG PATH:   2 NUMBER OF K POINTS:   41

  FROM K  (   2   2   0 ) TO K  (   3   0   0 ) WITH DENOMINATOR    6

  THE POSITION OF THE POINTS IS EXPRESSED IN UNITS        OF DENOMINATOR  240

 *******************************************************************************
```
 
 At the 75th qpoint:  
```
 DISPERSION K POINT NUMBER    75 COORD:  C( 113  14   0 )    WEIGHT:    1.

    MODES         EIGV          FREQUENCIES     IRREP
             (HARTREE**2)   (CM**-1)     (THZ)
    1-   1    0.4928E-05    487.1942   14.6057  (  1)
    2-   2    0.8383E-05    635.4696   19.0509  (  1)
    3-   3    0.9116E-05    662.6443   19.8656  (  1)
    4-   4    0.3933E-04   1376.3503   41.2619  (  1)
    5-   5    0.4131E-04   1410.6170   42.2892  (  1)
    6-   6    0.4439E-04   1462.3150   43.8391  (  1)

 MODES IN PHASE

 FREQ(CM**-1)    487.19    635.47    662.64   1376.35   1410.62   1462.32

 AT.   1 C  X     0.0001    0.0001    0.0001   -0.2006    0.1997    0.0072
            Y     0.0000   -0.0001    0.2040    0.0015   -0.0002   -0.1963
            Z     0.2041   -0.2007   -0.0001    0.0001    0.0001    0.0000
 AT.   2 C  X    -0.0001    0.0001   -0.0010    0.2041    0.1963    0.0003
            Y     0.0000   -0.0001    0.2006   -0.0004   -0.0077    0.1996
            Z     0.2007    0.2041    0.0001    0.0001   -0.0001   -0.0000 
```
